In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sfma import Data, Variable, SplineVariable, SplineGetter, SplinePriorGetter, UniformPrior, SFMAModel
import matplotlib.ticker as mticker

You have to run this script a couple different times: 
For the primary models of the paper, run the cells below this one six times with a different selected_cause_name specified each time:
    selected_cause_name = one of "All causes", "Cardiovascular diseases", "Maternal disorders", "Breast cancer", "Cervical cancer", "Ovarian cancer", "Uterine cancer"
    covariate_name = "sdi"
    measure_name = "dalys"
    version = "2spline"
    lineartails = True
    trimmed = True
    crossectional = False

For the sensitivity analyses, you'll want to run the script with variations to the other parameters six times each with a different selected_cause_name specified each time. Here are the options that are available at the moment (4/22/2025):
    covariate_name = "sdi" or "ldi"
    measure_name = "dalys"
    version = "2spline" or "3spline"
    crosssectional = True or False

In [ ]:
selected_cause_name = "Uterine cancer"
covariate_name = "sdi"
cause_subset = "top_and_sexspecific"
measure_name = "dalys"
version = "2spline"
lineartails = True
pct_trim = 0.05
crosssectional = False

In [83]:
if covariate_name != "sdi":
    version = version + "_" + covariate_name

if crosssectional:
    version = version + "_xsectional"

if lineartails == False:
    version = version + "_nonlineartails"

if pct_trim not in [0.05, 0]:
    version = version + "_" + str(round(pct_trim*100)) + "trim"
elif pct_trim == 0:
    version = version + "_notrim"
    
print(version)

2spline_ldi


In [ ]:
cause_label = selected_cause_name.replace(" ", "_").lower()

data_path = "filepath/" + covariate_name + "_" + cause_subset + "/" + measure_name + "_" + cause_label + "_compiled.csv"
output_dir = "filepath/"

# Load the data
data = pd.read_csv(data_path)
data.head()

,location_id,location_name,year_id,age_group_id,age_group_name,cause_id,cause_name,sex_id,sex,measure_name,metric_name,val,upper,lower,mean_predictor,upper_predictor,lower_predictor,ihme_loc_id,region_name,super_region_name
0,6,China,1990,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),Rate,0.000971,0.001259,0.000586,1955.405591,1955.405591,1955.405591,CHN,East Asia,"Southeast Asia, East Asia, and Oceania"
1,6,China,1991,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),Rate,0.000959,0.001246,0.000590,2032.771283,2032.771283,2032.771283,CHN,East Asia,"Southeast Asia, East Asia, and Oceania"
2,6,China,1992,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),Rate,0.000945,0.001207,0.000583,2128.378772,2128.378772,2128.378772,CHN,East Asia,"Southeast Asia, East Asia, and Oceania"
3,6,China,1993,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),Rate,0.000929,0.001208,0.000576,2247.259509,2247.259509,2247.259509,CHN,East Asia,"Southeast Asia, East Asia, and Oceania"
4,6,China,1994,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),Rate,0.000911,0.001182,0.000571,2389.603988,2389.603988,2389.603988,CHN,East Asia,"Southeast Asia, East Asia, and Oceania"


In [85]:
# define important variable names:
x = 'mean_predictor' # SDI, main predictor + spline variable
y = 'val' # outcome variable

# calculate the standard error of the outcome variable
data['log_se'] = (np.log(data['upper']) - np.log(data['lower'])) / (2 * 1.96)

# log outcome
data['log_val'] = np.log(data[y])

# Flip the outcome variable
data['neg_log_val'] = -data['log_val']

data.head()

,location_id,location_name,year_id,age_group_id,age_group_name,cause_id,cause_name,sex_id,sex,measure_name,...,lower,mean_predictor,upper_predictor,lower_predictor,ihme_loc_id,region_name,super_region_name,log_se,log_val,neg_log_val
0,6,China,1990,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),...,0.000586,1955.405591,1955.405591,1955.405591,CHN,East Asia,"Southeast Asia, East Asia, and Oceania",0.195197,-6.936932,6.936932
1,6,China,1991,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),...,0.000590,2032.771283,2032.771283,2032.771283,CHN,East Asia,"Southeast Asia, East Asia, and Oceania",0.190519,-6.949911,6.949911
2,6,China,1992,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),...,0.000583,2128.378772,2128.378772,2128.378772,CHN,East Asia,"Southeast Asia, East Asia, and Oceania",0.185528,-6.963976,6.963976
3,6,China,1993,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),...,0.000576,2247.259509,2247.259509,2247.259509,CHN,East Asia,"Southeast Asia, East Asia, and Oceania",0.188926,-6.981908,6.981908
4,6,China,1994,27,Age-standardized,435,Uterine cancer,2,Female,DALYs (Disability-Adjusted Life Years),...,0.000571,2389.603988,2389.603988,2389.603988,CHN,East Asia,"Southeast Asia, East Asia, and Oceania",0.185715,-7.001320,7.001320


In [86]:
# Setting model parameters
if covariate_name == "ldi": 
    data[x] = np.log(data[x])

# Number of knots
if "2spline" in version: 
    knots_version = [0, 0.33, 0.66, 1]
    degree_version = 2
elif "3spline" in version:
    knots_version = [0, 0.25, 0.5, 0.75, 1]
    degree_version = 3
else:
    raise ValueError("Invalid spline option specified.")

# type of tails
if "nonlineartails" in version: 
    tail_version = False
else:
    tail_version = True

if "xsectional" in version: 
    data = data[data['year_id'] == 2023]

In [87]:
nlocations = data['location_name'].nunique()
nyears = data['year_id'].nunique()
nages = data['age_group_name'].nunique()
print(f"Number of locations: {nlocations}")
print(f"Number of years: {nyears}")
print(f"Number of ages: {nages}")

Number of locations: 204
Number of years: 34
Number of ages: 1


In [88]:
# Group by 'year_id' and 'location_name' and check the size of each group
size_we_want = data['cause_name'].nunique()
group_sizes = data.groupby(['year_id', 'location_name']).size()

# Confirm that all groups have exactly 6 rows
if (group_sizes == size_we_want).all():
    print("Number of outcomes we want: " + str(size_we_want))
    print("All combinations of year_id and location_name have exactly the number of rows per outcome we want.")
else:
    print("Some combinations of year_id and location_name do not have a row for each outcome.")
    print(group_sizes[group_sizes != size_we_want])  # Print the problematic groups

Number of outcomes we want: 1
All combinations of year_id and location_name have exactly the number of rows per outcome we want.


In [89]:
num_rows = data.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of rows in the dataset: 6936


In [90]:
# create model object with log transformed data
model_data = Data(obs = 'neg_log_val',
            obs_se = 'log_se'
            )

variables = [
    SplineVariable(x,
                   spline=SplineGetter(knots = knots_version,
                                       degree = degree_version,
                                       r_linear = tail_version,
                                       l_linear = tail_version,
                                       knots_type = "rel_freq",
                                       include_first_basis = True))
]

model = SFMAModel(model_data, variables)
model.attach(data)

In [91]:
model.fit(verbose = True, outlier_pct = pct_trim,
          eta_options = {"method": "bounded", "bounds": [0.0, 100.0]}
          )

counter=  0, obj=8.84e+06, eta=1.00e+00, gamma=0.00e+00, error=1.00e+00
counter=  1, obj=5.74e+03, eta=8.57e-01, gamma=0.00e+00 error=7.11e+00
counter=  2, obj=5.72e+03, eta=8.04e-01, gamma=0.00e+00 error=5.25e-02
counter=  3, obj=5.72e+03, eta=7.83e-01, gamma=0.00e+00 error=2.10e-02
counter=  4, obj=5.72e+03, eta=7.75e-01, gamma=0.00e+00 error=8.71e-03
counter=  0, obj=4.64e+03, eta=7.75e-01, gamma=0.00e+00, error=1.00e+00
counter=  1, obj=4.32e+03, eta=5.92e-01, gamma=0.00e+00 error=1.83e-01
counter=  2, obj=4.27e+03, eta=5.32e-01, gamma=0.00e+00 error=5.91e-02
counter=  3, obj=4.26e+03, eta=5.11e-01, gamma=0.00e+00 error=2.19e-02


counter=  4, obj=4.26e+03, eta=5.02e-01, gamma=0.00e+00 error=8.55e-03
counter=  0, obj=4.07e+03, eta=5.02e-01, gamma=0.00e+00, error=1.00e+00
counter=  1, obj=4.01e+03, eta=4.71e-01, gamma=0.00e+00 error=5.90e-02
counter=  2, obj=4.00e+03, eta=4.59e-01, gamma=0.00e+00 error=1.15e-02
counter=  3, obj=4.00e+03, eta=4.55e-01, gamma=0.00e+00 error=4.37e-03
counter=  0, obj=3.95e+03, eta=4.55e-01, gamma=0.00e+00, error=1.00e+00
counter=  1, obj=3.94e+03, eta=4.47e-01, gamma=0.00e+00 error=3.25e-02
counter=  2, obj=3.94e+03, eta=4.44e-01, gamma=0.00e+00 error=2.79e-03
counter=  0, obj=3.92e+03, eta=4.44e-01, gamma=0.00e+00, error=1.00e+00
counter=  1, obj=3.92e+03, eta=4.42e-01, gamma=0.00e+00 error=1.28e-02
counter=  2, obj=3.92e+03, eta=4.41e-01, gamma=0.00e+00 error=8.46e-04


In [92]:
# see beta coefficients
model.get_beta_dict()

{'mean_predictor': array([7.82549314, 7.65678912, 7.68121794])}

In [ ]:
# calculate inefficiency scores
data['log_ineff_pkg'] = model.get_inefficiency()
data['ineff_pkg'] = np.exp(model.get_inefficiency())

# make predictions with mean covariate values
Y_hat_null = model.predict(data)

# add preedicted columns to dataframe: 
data['log_pred_null'] = -Y_hat_null
data['val_pred'] = np.exp(data['log_pred_null'])

# calculate absolute inefficiency
data['abs_ineff'] = data['val'] - data['val_pred']

# save outliers for plotting
data['outliers'] = model.weights == 0.0

# order DF by SDI for correct plotting
data = data.sort_values(x)

# create outlier filter (after sorting to make sure index is correct)
outliers = data['outliers'] == True

log_knots = (variables[0].spline.knots)

print((log_knots))

# save out data frame with frontier predictions, inefficiency scores
data.to_csv(f"{output_dir}{cause_label}_{version}_{measure_name}_inefficiencies.csv", index=False)

[ 5.44830632  8.36500258  9.5727877  12.42205322]
